In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('HousePricing').master('local[*]').getOrCreate()

In [3]:
sdf = spark.read.option('header', 'true').option('inferSchema', 'true').csv('file:////Users/giovanna/Documents/GitHub/pyspark/SparkCourse/realestate.csv')

In [8]:
sdf.columns

['No',
 'TransactionDate',
 'HouseAge',
 'DistanceToMRT',
 'NumberConvenienceStores',
 'Latitude',
 'Longitude',
 'PriceOfUnitArea']

In [10]:
from pyspark.ml.feature import VectorAssembler

vAss = VectorAssembler(inputCols=['TransactionDate',
                                    'HouseAge',
                                    'DistanceToMRT',
                                    'NumberConvenienceStores'],\
                       outputCol="features")
sdf_ass = vAss.transform(sdf).select('features', 'PriceOfUnitArea')

In [12]:
sdf_ass_v = sdf_ass.randomSplit([0.5,0.5])
sdf_ass_train = sdf_ass_v[0]
sdf_ass_test = sdf_ass_v[1]

In [19]:
from pyspark.ml.regression import DecisionTreeRegressor

dTree = DecisionTreeRegressor(featuresCol='features', labelCol='PriceOfUnitArea')
dTree_model = dTree.fit(sdf_ass_train)

In [21]:
predictions = dTree_model.transform(sdf_ass_test)

In [22]:
predictions.show(10)

+--------------------+---------------+------------------+
|            features|PriceOfUnitArea|        prediction|
+--------------------+---------------+------------------+
|[2012.667,1.1,329...|           51.8|             44.95|
|[2012.667,1.5,23....|           47.7|             49.85|
|[2012.667,3.1,383...|           56.2|             44.95|
|[2012.667,3.1,577...|           47.7|             44.95|
|[2012.667,5.7,90....|           53.5|             49.85|
|[2012.667,12.9,49...|           42.5|40.086842105263166|
|[2012.667,14.6,33...|           26.5|40.086842105263166|
|[2012.667,15.5,81...|           37.4|            36.475|
|[2012.667,20.2,21...|           22.8| 23.83333333333333|
|[2012.667,30.4,17...|           25.9|13.433333333333335|
+--------------------+---------------+------------------+
only showing top 10 rows

